# Road Following - Build TensorRT model for live demo

In this notebook, we will optimize the model we trained using TensorRT.

In [1]:
#############################################
# Load the trained model
#############################################

import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

#############################################
# load the trained weights
#############################################

model.load_state_dict(torch.load('best_steering_model_xy.pth'))

#############################################
# Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.
#############################################

device = torch.device('cuda')

#############################################
# TensorRT
#############################################
# Convert and optimize the model using torch2trt for faster inference with TensorRT

from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

#############################################
# Save the optimized model
#############################################

torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt.pth')